In [43]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [44]:
path = "./UJIIndoorLoc/children_13/sorted/00_sorted.csv"
train_df = pd.read_csv(path,header=0)
print(train_df.shape)
a = train_df.mean()
a["WAP004"]
for i in train_df.columns[:520]:
    if a[i]==-110:
        del train_df[i]
print(train_df.shape)
train_df["REF"] = pd.factorize(train_df["REF"])[0].astype(int)#将标签映射到顺序数字上
labels = train_df.REF.values
features = train_df.drop(columns=['TIMESTAMP','PHONEID','USERID','RELATIVEPOSITION',
                                'SPACEID','BUILDINGID','FLOOR','LATITUDE','LONGITUDE',
                                'BF','REF']).values
features,labels,features.shape

(1059, 531)
(1059, 130)


(array([[-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        ...,
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110]], dtype=int64),
 array([ 0,  0,  0, ..., 53, 53, 53]),
 (1059, 119))

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, X_test, Y_train, Y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.3,
                                                    random_state= 150,
                                                    #random_state：可以理解为随机数种子，主要是为了复现结果而设置
                                                    stratify=labels)
X_train,Y_train = shuffle(X_train, Y_train)

In [46]:
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
scale = StandardScaler().fit(X_train)
X_train_transform = scale.transform(X_train)
X_test_transform = scale.transform(X_test)
Y_train_encoded = to_categorical(Y_train)
Y_test_encoded = to_categorical(Y_test)

In [47]:
pca = PCA(n_components = 'mle').fit(X_train_transform)
X_train_pca = pca.transform(X_train_transform)
X_test_pca = pca.transform(X_test_transform)
X_train_pca,X_test_pca,X_train_pca.shape,X_test_pca.shape

(array([[ 5.46430151e+00, -2.92329050e-01, -1.61343356e+00, ...,
          2.51579567e-03, -1.68286308e-02, -4.14738771e-03],
        [-3.17241454e+00, -5.10103575e-01, -4.65543965e-01, ...,
         -1.34414956e-02, -4.68692510e-02, -8.93858182e-03],
        [-3.45671235e+00,  4.50341598e-01, -6.86193859e-01, ...,
          6.96549004e-02,  1.61558449e-01,  4.20847530e-02],
        ...,
        [-5.22581809e-02,  3.97019607e+00,  3.36846107e+00, ...,
          6.79768022e-02,  6.56483518e-01,  8.12273902e-02],
        [-5.07982592e+00, -8.07077319e-01, -1.27362947e+00, ...,
         -6.91104819e-03, -1.94075190e-02, -1.78677203e-02],
        [-3.95808177e+00, -8.39718935e-01, -5.61166290e-01, ...,
          2.52814587e-02, -6.65712902e-02, -9.21038198e-03]]),
 array([[-3.08200038e+00, -5.22166080e-01, -2.99465228e-01, ...,
          9.32222262e-03,  6.47675199e-02,  1.04951345e-02],
        [-5.53962153e+00,  6.75819602e-01,  1.26392194e+00, ...,
         -3.17614225e-02,  3.34110151e

In [48]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_shape=(111,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(54, activation='softmax'))
model.compile(optimizer=Adam(0.0002, 0.5), loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train_pca,Y_train_encoded,epochs=198,batch_size=32,validation_split=0.2,verbose=1)

Epoch 1/198
19/19 [==============================] - 1s 16ms/step - loss: 3.9969 - accuracy: 0.0321 - val_loss: 3.8755 - val_accuracy: 0.0604
Epoch 2/198
19/19 [==============================] - 0s 5ms/step - loss: 3.8284 - accuracy: 0.0693 - val_loss: 3.7592 - val_accuracy: 0.1007
Epoch 3/198
19/19 [==============================] - 0s 4ms/step - loss: 3.6844 - accuracy: 0.1503 - val_loss: 3.6381 - val_accuracy: 0.1611
Epoch 4/198
19/19 [==============================] - 0s 5ms/step - loss: 3.5345 - accuracy: 0.2162 - val_loss: 3.5077 - val_accuracy: 0.2349
Epoch 5/198
19/19 [==============================] - 0s 5ms/step - loss: 3.3690 - accuracy: 0.2889 - val_loss: 3.3618 - val_accuracy: 0.3221
Epoch 6/198
19/19 [==============================] - 0s 5ms/step - loss: 3.1874 - accuracy: 0.3581 - val_loss: 3.1966 - val_accuracy: 0.3557
Epoch 7/198
19/19 [==============================] - 0s 4ms/step - loss: 2.9874 - accuracy: 0.3986 - val_loss: 3.0141 - val_accuracy: 0.4027
Epoch 8/198


Epoch 59/198
19/19 [==============================] - 0s 4ms/step - loss: 0.2396 - accuracy: 0.9679 - val_loss: 1.0514 - val_accuracy: 0.7248
Epoch 60/198
19/19 [==============================] - 0s 5ms/step - loss: 0.2331 - accuracy: 0.9662 - val_loss: 1.0567 - val_accuracy: 0.7248
Epoch 61/198
19/19 [==============================] - 0s 5ms/step - loss: 0.2272 - accuracy: 0.9679 - val_loss: 1.0530 - val_accuracy: 0.7248
Epoch 62/198
19/19 [==============================] - 0s 5ms/step - loss: 0.2207 - accuracy: 0.9696 - val_loss: 1.0718 - val_accuracy: 0.7315
Epoch 63/198
19/19 [==============================] - 0s 3ms/step - loss: 0.2154 - accuracy: 0.9696 - val_loss: 1.0565 - val_accuracy: 0.7315
Epoch 64/198
19/19 [==============================] - 0s 4ms/step - loss: 0.2110 - accuracy: 0.9645 - val_loss: 1.0717 - val_accuracy: 0.7248
Epoch 65/198
19/19 [==============================] - 0s 5ms/step - loss: 0.2044 - accuracy: 0.9696 - val_loss: 1.0702 - val_accuracy: 0.7315
Epoch 

19/19 [==============================] - 0s 4ms/step - loss: 0.0669 - accuracy: 0.9899 - val_loss: 1.1949 - val_accuracy: 0.7383
Epoch 117/198
19/19 [==============================] - 0s 3ms/step - loss: 0.0648 - accuracy: 0.9882 - val_loss: 1.2074 - val_accuracy: 0.7315
Epoch 118/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0650 - accuracy: 0.9899 - val_loss: 1.2177 - val_accuracy: 0.7315
Epoch 119/198
19/19 [==============================] - 0s 3ms/step - loss: 0.0644 - accuracy: 0.9916 - val_loss: 1.2037 - val_accuracy: 0.7315
Epoch 120/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0621 - accuracy: 0.9916 - val_loss: 1.2242 - val_accuracy: 0.7248
Epoch 121/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0633 - accuracy: 0.9916 - val_loss: 1.2084 - val_accuracy: 0.7383
Epoch 122/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0607 - accuracy: 0.9899 - val_loss: 1.2174 - val_accuracy: 0.7315
Epoch 123/198

19/19 [==============================] - 0s 5ms/step - loss: 0.0238 - accuracy: 0.9966 - val_loss: 1.3523 - val_accuracy: 0.7315
Epoch 174/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0246 - accuracy: 0.9966 - val_loss: 1.3502 - val_accuracy: 0.7315
Epoch 175/198
19/19 [==============================] - 0s 3ms/step - loss: 0.0240 - accuracy: 0.9983 - val_loss: 1.3561 - val_accuracy: 0.7315
Epoch 176/198
19/19 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.9966 - val_loss: 1.3723 - val_accuracy: 0.7315
Epoch 177/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0229 - accuracy: 0.9983 - val_loss: 1.3669 - val_accuracy: 0.7315
Epoch 178/198
19/19 [==============================] - 0s 3ms/step - loss: 0.0218 - accuracy: 0.9966 - val_loss: 1.3683 - val_accuracy: 0.7248
Epoch 179/198
19/19 [==============================] - 0s 4ms/step - loss: 0.0216 - accuracy: 0.9966 - val_loss: 1.3794 - val_accuracy: 0.7383
Epoch 180/198

In [49]:
loss,acc = model.evaluate(X_test_pca,Y_test_encoded)
loss,acc

10/10 [==============================] - 0s 3ms/step - loss: 1.7013 - accuracy: 0.7233


(1.7013012170791626, 0.7232704162597656)